#Library import

In [ ]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import networkx as nx
import csv

from google.colab import drive
import os
import csv

import mmap
import string
import spacy

import pickle as pkl
#from sklearn.externals import joblib
from numba import jit

# package imports
import gc
import time
import warnings

from numba import jit
warnings.filterwarnings("ignore")


In [ ]:
!pip install surprise

In [ ]:
!pip install surprise
from surprise import SVD
from surprise import SVDpp
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import GridSearchCV
from surprise import Dataset
from surprise import BaselineOnly


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Thesis/Data')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#data processing

In [ ]:
df_all = pd.read_csv('amazon_reviews_us_Books_v1_02.tsv',sep='\t', error_bad_lines=False)
df_all.head(3)

b'Skipping line 1680001: expected 15 fields, saw 22\n'


,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,12076615,RQ58W7SMO911M,0385730586,122662979,Sisterhood of the Traveling Pants (Book 1),Books,4.0,2.0,3.0,N,N,this book was a great learning novel!,this boook was a great one that you could lear...,2005-10-14
1,US,12703090,RF6IUKMGL8SF,0811828964,56191234,The Bad Girl's Guide to Getting What You Want,Books,3.0,5.0,5.0,N,N,Fun Fluff,If you are looking for something to stimulate ...,2005-10-14
2,US,12257412,R1DOSHH6AI622S,1844161560,253182049,"Eisenhorn (A Warhammer 40,000 Omnibus)",Books,4.0,1.0,22.0,N,N,this isn't a review,never read it-a young relative idicated he lik...,2005-10-14


In [ ]:
df_all.rename(columns = {'customer_id':'user_id','product_id':'book_id'}, inplace = True)

In [ ]:
# hold out last review
ratings_holdout = df_all.groupby(['user_id'],sort=False).tail(5)
#ratings_holdout.shape
index_holdout=ratings_holdout.index.values.tolist()
test_df = df_all.index.isin(index_holdout)
ratings_train = df_all[~test_df]
print('There are {0} rows, {1} columns in training set.'.format(
    ratings_train.shape[0], ratings_train.shape[1]))
print('There are {0} rows, {1} columns in holdout set.'.format(
    ratings_holdout.shape[0], ratings_holdout.shape[1]))

There are 860315 rows, 15 columns in training set.
There are 2245055 rows, 15 columns in holdout set.


In [ ]:
df_train = ratings_train.copy()
df_test = ratings_holdout.copy()
trainset = df_train.reset_index(drop=True)
testset = df_test.reset_index(drop=True)

In [ ]:
reader = Reader(rating_scale = (0.0, 5.0))
train_data = Dataset.load_from_df(trainset[['user_id', 'book_id','star_rating']], reader)
test_data = Dataset.load_from_df(testset[['user_id', 'book_id','star_rating']], reader)

train_sr = train_data.build_full_trainset()
test_sr_before = test_data.build_full_trainset()
test_sr = test_sr_before.build_testset()

In [ ]:
import gc
#del [[ratings_holdout]]
#del [[index_holdout]]  
#del [[test_df]]  
#del [[ratings_train]]  
gc.collect()


96

In [ ]:
books =  pd.read_csv('link_pred_data.csv')

In [ ]:
books.head(5)

,Unnamed: 0,bookid,main_genre
0,0,62501526,"['PhotoEssays(Books)', 'Customs&TraditionsSoci..."
1,1,131175289,"['SpanishLanguageInstruction(Books)', 'Cultura..."
2,2,140050337,"['ContemporaryLiterature&Fiction', '']"
3,3,156026996,"['DealsinBooks', 'AmericanLiterature(Books)', ..."
4,4,198645155,"['Dictionaries(Books)', 'SpanishLanguageInstru..."


In [ ]:
books.dropna(inplace = True)





#Metrics

##HitRate

In [ ]:
def hit_rate(user_id,df,recommend_list):
  hit_df = pd.DataFrame()
  hit_df = df[(df['user_id']==user_id)][['user_id', 'book_id']]
  hit=0.
  hit_list = pd.merge(
      recommend_list,hit_df,on=[ 'book_id'],how='inner')
  if (len(hit_list)>=1):
    # print(user_id+" no of relevent business "+str(len(hit_list)))
    hit=1
  return (user_id, hit)

In [ ]:
def total_hit_rate(reclist,df,num_of_usrs,num_of_rec,sort_flag=False):
  user_df = df['user_id'].unique().tolist()
  result_hit = []
  for i in tqdm(user_df[:num_of_usrs]):
    # print(i)
    if reclist == "Linkedprediction":
      # print('LP')
      recommend_list = get_n_recommendations(i,df_all,emb_df,emb,num_of_rec)
    elif reclist == "Collaborative_Filter_svd":
      # print('SVD')
      recommend_list =  get_top_n_svd(i,df_all, n=num_of_rec)
    elif reclist == "Collaborative_Filter_svdpp":
      # print('SvD')
      recommend_list = get_top_n_svdpp(i,df_all, n=num_of_rec)
    elif reclist == "hybrid_weighted":
      # print('Hybrid')
      recommend_list = get_top_n_hybrid_mixed_weighted_model(i, '764223259',df_all, books, num_of_rec, False)
    hit = hit_rate(i,df,recommend_list)
    result_hit.append(hit[1])
  print(
      '{0} model total hit rate for {1} users with {2} recommended list is {3}'\
      .format(reclist,num_of_usrs,num_of_rec,
      round(sum(result_hit)/num_of_usrs,4)))
  return (sum(result_hit)/num_of_usrs)

In [ ]:
hit = total_hit_rate("Linkedprediction",testset,100,10,False)
print('Hit_rate for testset is {0}'.format(round(hit,4)))

  0%|          | 0/100 [00:00<?, ?it/s]

Linkedprediction model total hit rate for 100 users with 10 recommended list is 0.0
Hit_rate for testset is 0.0


#Collaborative filtering

##SVD

In [ ]:
#Loading saved model
with open('SVD_rating_model.p','rb') as fp:
  cf_svd_rating_model = pkl.load(fp)

In [ ]:
def get_top_n_svd(user_id,df, n=10,sort_flag=False):
 
  user = df[['book_id', 'product_title']]
  user = user.drop_duplicates(subset=['book_id'])
  user['user_id'] = user_id
  user['predicted_Score'] = user['book_id'].apply(
      lambda x: cf_svd_rating_model.predict(user_id, x).est)
  # user['Estimate_Score_minmax'] = ((user['predicted_Score'] - user['predicted_Score'].min(axis=0)) / (user['predicted_Score'].max(axis=0) - user['predicted_Score'].min(axis=0)))*5
  user = user.sort_values('predicted_Score', ascending=sort_flag)
  return user[:n]

In [ ]:
top_n_svd = get_top_n_svd("45586816",df_all, n=100)

In [ ]:
top_n_svd.head(15)

,book_id,product_title,user_id,predicted_Score
141275,1861060408,The Rose of Enchantment,45586816,4.999951
8197,0553582216,By Possession (Medievals),45586816,4.935000
175040,0972660208,Passing the Flame: A Beadmaker's Guide to Deta...,45586816,4.876449
118003,0451199707,The Stone Maiden,45586816,4.859113
431304,1413707203,Our Stories of Domestic Abuse,45586816,4.852967
222484,0929292936,Finding A Job God's Way: Moving into the HOV L...,45586816,4.851696
22034,0735712794,Photoshop Masking & Compositing,45586816,4.850419
58366,0553379887,Pain Free: A Revolutionary Method for Stopping...,45586816,4.849515
58749,0609804618,Our Dumb Century: The Onion Presents 100 Years...,45586816,4.848397
145921,0380753022,Gentle Rogue (Malory-Anderson Family),45586816,4.847580


##SVD++

In [ ]:
#Loading saved model
with open('SVDpp_rating_model.p','rb') as fp:
  cf_svdpp_rating_model = pkl.load(fp)

In [ ]:
def get_top_n_svdpp(user_id,df, n=10,sort_flag=False):
  user = df[['book_id', 'product_title']]
  user = user.drop_duplicates(subset=['book_id'])
  user['user_id'] = user_id
  user['predicted_Score'] = user['book_id'].apply(
      lambda x: cf_svdpp_rating_model.predict(user_id, x).est)
  # user['Estimate_Score_minmax'] = ((user['predicted_Score'] - user['predicted_Score'].min(axis=0)) / (user['predicted_Score'].max(axis=0) - user['predicted_Score'].min(axis=0)))*5
  user = user.sort_values('predicted_Score', ascending=sort_flag)
  return user[:n]

In [ ]:
top_n_svdpp = get_top_n_svdpp("45586816",df_all, n=10)

In [ ]:
top_n_svdpp.head(10)

,book_id,product_title,user_id,predicted_Score
201318,0743235576,Storm on the Horizon: Khafji--The Battle That ...,45586816,4.950433
8197,0553582216,By Possession (Medievals),45586816,4.934910
141275,1861060408,The Rose of Enchantment,45586816,4.910850
29533,0553567683,Adam's Fall (Mason Sisters),45586816,4.909661
208748,0321245784,Real World Adobe Photoshop CS,45586816,4.897897
58749,0609804618,Our Dumb Century: The Onion Presents 100 Years...,45586816,4.896837
58366,0553379887,Pain Free: A Revolutionary Method for Stopping...,45586816,4.876467
34372,0761129235,One Small Step Can Change Your Life: The Kaize...,45586816,4.862245
10780,0345298349,Gorky Park (Arkady Renko Novels),45586816,4.860626
13006,0974070505,Hood Rich,45586816,4.859819


#Linked prediction based filtering

In [ ]:
!pip install node2vec

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
from node2vec import Node2Vec as n2v

In [ ]:
#loading pickel file
pickle_off = open ("genre_mdl.pkl", "rb")
genre_mdl = pkl.load(pickle_off)
print(genre_mdl)
#loading pickel file
pickle_off_nx = open ("genre_nx.pkl", "rb")
emb = pkl.load(pickle_off_nx)

Word2Vec(vocab=502, size=15, alpha=0.025)


In [ ]:
# create embeddings dataframe
emb_df = (
    pd.DataFrame(
        [genre_mdl.wv.get_vector(str(n)) for n in emb.nodes()],
        index = emb.nodes
    )
)


In [ ]:
emb_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
006015828X,-0.513200,-0.022362,0.103371,0.110137,-0.215115,0.149876,-0.084862,0.039836,-0.062740,-0.527484,-0.494363,0.375896,-0.281954,0.104045,0.055629
006093204X,-0.699362,-0.230076,0.220996,-0.085450,-0.208365,0.199538,-0.161255,0.024018,-0.207925,-0.342505,-0.490684,0.131813,-0.200679,0.060244,0.042061
006207749X,-0.596464,-0.241948,0.117779,0.252982,-0.200747,0.034862,-0.121703,0.099875,-0.124503,-0.324678,-0.366697,0.260217,-0.518743,0.133528,0.122716
006232280X,-0.461314,-0.240518,0.123070,0.101420,-0.249884,-0.040556,-0.174842,-0.051184,-0.193530,-0.406139,-0.473576,0.197221,-0.261962,0.261374,0.222043
030734150X,-0.498475,-0.239512,0.111093,0.174231,-0.230540,0.165347,-0.183793,-0.094505,-0.112577,-0.583129,-0.252021,0.200525,-0.204804,0.275184,0.394711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985819200,-0.557301,-0.294002,-0.035523,0.146767,-0.123977,0.043547,-0.008197,-0.137225,-0.171022,-0.547444,-0.332108,0.269265,-0.348411,0.093533,0.037984
9881613973,-0.405107,-0.205813,0.147937,-0.011149,-0.310248,0.024445,-0.232619,-0.063868,-0.075011,-0.555050,-0.498919,0.149528,-0.314270,0.070374,0.080510
9881998468,-0.588646,-0.258887,0.141371,0.078448,-0.193297,0.056064,-0.166258,-0.045429,-0.089570,-0.318640,-0.448813,0.237115,-0.399486,0.034252,0.207134
996762191,-0.472148,-0.251243,0.174761,0.066890,-0.134122,0.049480,-0.263999,-0.064987,-0.146236,-0.432304,-0.383003,0.339351,-0.398869,-0.011329,0.215334


In [ ]:
def predict_links(G, df, book_id, N):
    # print('predict_links')
    if book_id not in df.index:
      book_id = 'B0014IQFDS'
    books = df[df.index == book_id]
    
    all_nodes = G.nodes()
    other_nodes = [n for n in all_nodes if n not in list(G.adj[book_id]) + [book_id]]
    other_books = df[df.index.isin(other_nodes)]
    
    sim = cosine_similarity(books, other_books)[0].tolist()
    idx = other_books.index.tolist()
    
    idx_sim = dict(zip(idx, sim))
    idx_sim = sorted(idx_sim.items(), key=lambda x: x[1], reverse=True)
    
    similar_books = idx_sim[:N]
    books = [art for art in similar_books]
    return (books)

In [ ]:
predict_links(G = emb, df = emb_df, book_id = '764223259', N = 10)

predict_links


[('1624140599', 0.7501553893089294),
 ('1454908661', 0.7302678823471069),
 ('1628737123', 0.7219510078430176),
 ('9814346470', 0.7069754004478455),
 ('967089735', 0.6720024347305298),
 ('871975793', 0.671951949596405),
 ('982293968', 0.6661361455917358),
 ('9814561436', 0.664014995098114),
 ('824817788', 0.6612359285354614),
 ('578135213', 0.660489022731781)]

In [ ]:
def get_n_recommendations(user_id, df_all, embd_df, graf, no_of_recommendations):
  a = df_all[df_all['user_id']==int(user_id)]
  a.sort_values(by='star_rating',ascending=False)
  bk_id = str(a['book_id'].iloc[0])
  result_df = pd.DataFrame()
  book_id = df_all[df_all['user_id']==int(user_id)][['book_id']]
  sim_books = predict_links(G = emb, df = emb_df, book_id = bk_id , N = 10)
  book_ids = [x[0] for x in sim_books]
  result_df['book_id'] = [x[0] for x in sim_books]
  result_df['similarity'] = [x[1] for x in sim_books]
  result_df['Estimate_Score_minmax'] = ((result_df['similarity'] - result_df['similarity'].min(axis=0)) / (result_df['similarity'].max(axis=0) - result_df['similarity'].min(axis=0)))*5
  result_df['user_id'] = user_id
  result_df = result_df.sort_values('similarity',ascending=False)
  return result_df

In [ ]:
output_df = get_n_recommendations("12703090",df_all,emb_df,emb,10)

In [ ]:
output_df.head(10)

,book_id,similarity,Estimate_Score_minmax,user_id
0,764223259,0.536014,5.000000,12703090
1,345440900,0.531790,4.730311,12703090
2,1843100606,0.457712,0.000000,12703090


#Hybrid model

In [ ]:
def get_top_n_hybrid_mixed_weighted_model(user_id, bk_id, merged_df, books,no_of_recommendations,sort_flag=False):
  try:
    #LP  
    sim_books = predict_links(G = emb, df = emb_df, book_id = bk_id, N = 10)
    book_ids = [x[0] for x in sim_books]
    result_df_1 = pd.DataFrame()
    
    result_df_1['book_id'] = [x[0] for x in sim_books]
    result_df_1['similarity'] = [x[1] for x in sim_books]
    result_df_1['Estimate_Score_minmax'] = ((result_df_1['similarity'] - result_df_1['similarity'].min(axis=0)) / (result_df_1['similarity'].max(axis=0) - result_df_1['similarity'].min(axis=0)))*5
    result_df_1['user_id'] = user_id
    result_df_1 = result_df_1[['user_id','book_id','Estimate_Score_minmax']]
    result_df_1['Estimate_Score_minmax_1'] = result_df_1['Estimate_Score_minmax'] * 0.24
    result_df_1['Estimate_Score_1'] = result_df_1['Estimate_Score_minmax_1'].replace(np.nan, 0)
    result_df_1 = result_df_1[['user_id','book_id','Estimate_Score_1']]
    # print(len(result_df_1))
      
    #CF
    user = df_all[[ 'book_id', 'product_title']]
    user = user.drop_duplicates(subset=['book_id'])
    user['user_id'] = user_id
    user['Estimate_Score_svd'] = user['book_id'].apply(lambda x: cf_svd_rating_model.predict(user_id, x).est)
    user['Estimate_Score_svdpp'] = user['book_id'].apply(lambda x: cf_svdpp_rating_model.predict(user_id, x).est)
    user['Estimate_Score_svd_mm'] = ((user['Estimate_Score_svd'] - user['Estimate_Score_svd'].min(axis=0)) / (user['Estimate_Score_svd'].max(axis=0) - user['Estimate_Score_svd'].min(axis=0)))*5
    user['Estimate_Score_svdpp_mm'] = ((user['Estimate_Score_svdpp'] - user['Estimate_Score_svdpp'].min(axis=0)) / (user['Estimate_Score_svdpp'].max(axis=0) - user['Estimate_Score_svdpp'].min(axis=0)))*5 
    user['Estimate_Score_2'] = user['Estimate_Score_svd_mm']*0.38
    user['Estimate_Score_3'] = user['Estimate_Score_svdpp_mm']*0.38
      # print(len(user)) 
      # user = user.sort_values('Estimate_Score', ascending=sort_flag)
    result = pd.merge(result_df_1,user,on=['user_id','book_id'],how='outer')
    result['Estimate_Score_1'] = result['Estimate_Score_1'].replace(np.nan, 0)
    result['Estimate_Score_2'] = result['Estimate_Score_2'].replace(np.nan, 0)
    result['Weighted_Estimated_Score'] = result['Estimate_Score_1'] + result['Estimate_Score_2'] + result['Estimate_Score_3']
    result = result[['user_id','book_id','Weighted_Estimated_Score']]
    result = result.sort_values('Weighted_Estimated_Score', ascending=sort_flag)
      # result = result.sort_values('Estimate_Score', ascending=sort_flag).drop_duplicates(subset=['user_id', 'business_id'], keep='last')
  except IndexError:
      print("error: "+str(user_id))
      pass

  return result[:no_of_recommendations]



In [ ]:
# %%timeit
top_n = get_top_n_hybrid_mixed_weighted_model("45586816", '764223259',df_all, books, 100, False)

In [ ]:
top_n

,user_id,book_id,Weighted_Estimated_Score
83149,45586816,1861060408,3.773234
7009,45586816,0553582216,3.750960
41292,45586816,0609804618,3.673823
41067,45586816,0553379887,3.660712
107824,45586816,0743235576,3.646174
...,...,...,...
8982,45586816,0596006187,3.520564
137552,45586816,0974869406,3.520474
21788,45586816,0345298063,3.519914
169937,45586816,1570671656,3.519524


In [ ]:
%who_ls DataFrame

['books',
 'df_all',
 'df_test',
 'df_train',
 'emb_df',
 'output_df',
 'ratings_holdout',
 'ratings_train',
 'testset',
 'top_n',
 'top_n_svd',
 'top_n_svdpp',
 'trainset']

##Evaluation

In [ ]:
hit = total_hit_rate("Collaborative_Filter_svd",df_all,10,100,False)
print('Hit_rate for testset is {0}'.format(round(hit,4)))

  0%|          | 0/10 [00:00<?, ?it/s]

Collaborative_Filter_svd model total hit rate for 10 users with 100 recommended list is 0.1
Hit_rate for testset is 0.1


In [ ]:
hit = total_hit_rate("Collaborative_Filter_svdpp",df_all,10,100,False)
print('Hit_rate for testset is {0}'.format(round(hit,4)))

  0%|          | 0/10 [00:00<?, ?it/s]

Collaborative_Filter_svdpp model total hit rate for 10 users with 100 recommended list is 0.1
Hit_rate for testset is 0.1


In [ ]:
hit = total_hit_rate("hybrid_weighted",df_all,10,100,False)
print('Hit_rate for testset is {0}'.format(round(hit,4)))

  0%|          | 0/10 [00:00<?, ?it/s]

hybrid_weighted model total hit rate for 10 users with 100 recommended list is 0.1
Hit_rate for testset is 0.1


In [ ]:
hit = total_hit_rate("Linkedprediction",df_all,10,100,False)
print('Hit_rate for testset is {0}'.format(round(hit,4)))

  0%|          | 0/10 [00:00<?, ?it/s]

Linkedprediction model total hit rate for 10 users with 100 recommended list is 0.0
Hit_rate for testset is 0.0
